In [1]:
import torch

payload = '/home/jay/.jupyter/rl/URL_baselines_with_SMM/models/states/maze/edl/1/snapshot_4000000.pt'

current = torch.load(payload, map_location=torch.device('cpu'))

In [2]:
print(current)

{'agent': <agent.edl.EDLAgent object at 0x7f31044fb3d0>, '_global_step': 4000000, '_global_episode': 90000}


In [3]:
agent = current['agent']

In [4]:
agent.skill_dim

80

In [5]:
goals = list()
with torch.no_grad():
    for episode in range(agent.skill_dim):
        # print(agent.vae.get_centroids(torch.tensor(episode)))
        goals.append(agent.vae.get_centroids(torch.tensor(episode)))

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipdb
from sklearn.neighbors import KernelDensity

plot_colors = ['blue', 'red', 'green', 'brown', 'orange', 'pink', 'rebeccapurple', 'olive', 'purple', 'cyan', 
               'black', 'maroon', 'yellow', 'navy', 'darkgreen', 'lavender', 'peru', 'fuchsia', 'slateblue', 'oldlace',
               'darkviolet', 'silver', 'lightcoral', 'mistyrose', 'salmon', 'tomato', 'chocolate', 'bisque', 'burlywood', 'tan',
               'darkgoldenrod', 'darkkhaki', 'olivedrab', 'darkseagreen', 'lightgreen', 'limegreen', 'lime', 'aquamarine', 'turquoise', 'teal',
               'deepskyblue', 'steelblue', 'dodgerblue', 'cornflowerblue', 'royalblue', 'mediumpurple', 'blueviolet', 'indigo', 'plum', 'palevioletred']

In [7]:
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

import os

os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'
os.environ['MUJOCO_GL'] = 'egl'

from pathlib import Path

import ipdb
import hydra
import numpy as np
import torch
import wandb
import imageio
from collections import OrderedDict
from tqdm import tqdm

import utils
import omegaconf
from logger import Logger
from replay_buffer import ReplayBufferStorage, make_replay_loader
from video import TrainVideoRecorder, VideoRecorder
import envs.make_maze as make_maze
from dm_env import specs

torch.backends.cudnn.benchmark = True

train_env = make_maze.make('square_bottleneck', 50)

In [8]:

train_env.plot_trajectory(trajectory = dict(), 
                          save_dir = '/home/jay/.jupyter/rl/URL_baselines_with_SMM/tmp.png',
                          step = 0,
                          use_wandb=False,
                          goal = goals)

No handles with labels found to put in legend.


<Figure size 432x288 with 0 Axes>

In [10]:
for i in torch.sort(torch.stack(goals), dim=0):
    print(i)

tensor([[[ 0.2381, -0.3377]],

        [[ 0.2601,  0.0915]],

        [[ 2.5573,  0.2046]],

        [[ 2.7720,  0.3714]],

        [[ 3.6820,  0.3852]],

        [[ 3.7176,  1.7744]],

        [[ 4.2625,  2.2904]],

        [[ 4.3572,  2.3359]],

        [[ 4.5117,  2.9223]],

        [[ 4.6540,  3.0767]],

        [[ 4.6541,  3.0818]],

        [[ 4.6563,  3.0828]],

        [[ 4.6565,  3.0863]],

        [[ 4.6568,  3.0883]],

        [[ 4.6586,  3.0889]],

        [[ 4.6587,  3.0902]],

        [[ 4.6603,  3.0903]],

        [[ 4.6605,  3.0904]],

        [[ 4.6611,  3.0914]],

        [[ 4.6617,  3.0917]],

        [[ 4.6618,  3.0920]],

        [[ 4.6618,  3.0922]],

        [[ 4.6619,  3.0929]],

        [[ 4.6635,  3.0935]],

        [[ 4.6637,  3.0941]],

        [[ 4.6639,  3.0956]],

        [[ 4.6639,  3.0958]],

        [[ 4.6644,  3.0963]],

        [[ 4.6645,  3.0967]],

        [[ 4.6645,  3.0969]],

        [[ 4.6649,  3.0970]],

        [[ 4.6650,  3.0975]],

        